# AF2Seq

In [ ]:
from af2seq import GradientDesign

In [ ]:
target = 'path/to/target'

# Gradient based design

In [ ]:
# Assign the class
d = GradientDesign(datadir='path/to/weights',output_path='path/to/output/directory')

In [ ]:
# Settings

# Set the weights on the loss function
loss = {"fape": 1.0, "ptm": 0.2, "plddt": 0.2}

# Design settings
d.design(target,
         start_seq=None,
         iterations=500,
         lr=1e-3,             # Learning rate to start with (uses ADAM)
         chains = None,       # Chains we want to design.
         aa_mask=['C'],       # add here which amino acids you wish to mask for updating
         fix_pos=None,        # Position mask, give the index of amino acids of the target not to update (only works for ptm model for now!)
         recycles=0,          # number of recycles
         clampval=0.0,        # weight of the clamping of the FAPE loss.
         modeltype='ptm',
         mode = 'gd',
         loss = loss)

In [ ]:
print(d)

In [ ]:
from af2seq import plotting

In [ ]:
plotting.loss_plot(d)

In [ ]:
plotting.plot_pred(d)

# Running solubleMPNN example

Run proteinMPNN as you would run normally. However, to use the soluble weights the flag *-use_soluble_model* needs to be added when running *protein_mpnn_run.py*.

For more information see: https://github.com/dauparas/ProteinMPNN

Example submission script:

In [ ]:
#!/bin/bash
#SBATCH --job-name=mpnn_batch_1.job
#SBATCH --cpus-per-task 1
#SBATCH --mem 64gb
#SBATCH --partition=gpu
#SBATCH --qos=gpu
#SBATCH --gres=gpu:1
#SBATCH -t 1:0:0

folder_with_pdbs="pdbs"

output_dir="outputs"
if [ ! -d $output_dir ]
then
	    mkdir -p $output_dir
fi


path_for_parsed_chains=$output_dir"/parsed_pdbs.jsonl"
path_for_assigned_chains=$output_dir"/assigned_pdbs.jsonl"
path_for_fixed_positions=$output_dir"/fixed_pdbs.jsonl"

chains_to_design="A"
fixed_positions="1,2,3,4,5,6,7,8,9,10"

python /ProteinMPNN/helper_scripts/parse_multiple_chains.py --input_path=$folder_with_pdbs --output_path=$path_for_parsed_chains

python /ProteinMPNN/helper_scripts/make_fixed_positions_dict.py --input_path=$path_for_parsed_chains --output_path=$path_for_fixed_positions --position_list "$fixed_positions"

python /ProteinMPNN/protein_mpnn_run.py \
	--jsonl_path $path_for_parsed_chains \
	--chain_id_jsonl $path_for_assigned_chains \
	--fixed_positions_jsonl $path_for_fixed_positions \
	--tied_positions_jsonl $path_for_tied_positions \
	--out_folder $output_dir \
	--num_seq_per_target 10 \
	--sampling_temp "0.2" \
	--seed 37 \
	--batch_size 1 \
	--use_soluble_model \
	--omit_AAs "XC" \
	--model_name "v_48_020"